In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from PIL import Image

In [2]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from google.colab.patches import cv2_imshow
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Skin_Cancer_Detection/Data/data.csv')

# Load the dataset using OpenCV
# image_paths_bcc = ['/content/gdrive/MyDrive/Skin_Cancer_Detection/Data/bcc/' + image + '.jpg' for image in df.image_id[:514]]
image_paths_akeic = ['/content/gdrive/MyDrive/Skin_Cancer_Detection/Data/akiec/' + image + '.jpg' for image in df.image_id[514:]]
# image_paths = image_paths_bcc + image_paths_akeic
# images = [cv2.imread(path) for path in image_paths]

In [3]:
df['label'] = df['dx'].map({'bcc': 0, 'akiec':1})
df.head()

,Unnamed: 0,lesion_id,image_id,dx,dx_type,age,sex,localization,label
0,2462,HAM_0000781,ISIC_0028155,bcc,histo,50.0,male,back,0
1,2463,HAM_0003220,ISIC_0034093,bcc,histo,80.0,female,face,0
2,2464,HAM_0007141,ISIC_0029230,bcc,histo,50.0,male,trunk,0
3,2465,HAM_0007141,ISIC_0031513,bcc,histo,50.0,male,trunk,0
4,2466,HAM_0007009,ISIC_0032384,bcc,histo,50.0,male,back,0


In [11]:
import os

benign_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/Data_Skin_Cancer/data/train/benign'
malignant_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/Data_Skin_Cancer/data/train/malignant'

benign_list = os.listdir(benign_path)
malignant_list = os.listdir(malignant_path)

image_benign = [f for f in benign_list if f.endswith('.jpg') or f.endswith('.png')]
image_malignant = [f for f in malignant_list if f.endswith('.jpg') or f.endswith('.png')]
print(image_benign[0])
image_benign_path = [benign_path + '/' + image for image in image_benign]
print(image_benign_path[0])
image_malignant_path = [malignant_path + '/' + image for image in image_malignant]

1442.jpg
/content/gdrive/MyDrive/Skin_Cancer_Detection/Data_Skin_Cancer/data/train/benign/1442.jpg


In [12]:
import cv2
for i, path in enumerate(image_benign_path):
    # Load the input image
    img = cv2.imread(path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding to segment the image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological operations to clean up the binary image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours and find the contour with the maximum area
    max_area = 0
    max_cnt = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # If the area of a contour is greater than the current max, update the max
        if area > max_area:
            max_area = area
            max_cnt = cnt

    # Crop the bounding box with the maximum area from the input image and save it to a new file
    if max_cnt is not None:
        x, y, w, h = cv2.boundingRect(max_cnt)
        cropped = img[y:y+h, x:x+w]
        output_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign/' + image_benign[i]
        cv2.imwrite(output_path, cropped)


In [13]:
import cv2
for i, path in enumerate(image_malignant_path):
    # Load the input image
    img = cv2.imread(path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding to segment the image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological operations to clean up the binary image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours and find the contour with the maximum area
    max_area = 0
    max_cnt = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # If the area of a contour is greater than the current max, update the max
        if area > max_area:
            max_area = area
            max_cnt = cnt

    # Crop the bounding box with the maximum area from the input image and save it to a new file
    if max_cnt is not None:
        x, y, w, h = cv2.boundingRect(max_cnt)
        cropped = img[y:y+h, x:x+w]
        output_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/malignant/' + image_malignant[i]
        cv2.imwrite(output_path, cropped)


In [43]:
# Viet function predict mot anh
def crop_object_detection(file_path):
    img = cv2.imread(file_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding to segment the image
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological operations to clean up the binary image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours and find the contour with the maximum area
    max_area = 0
    max_cnt = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # If the area of a contour is greater than the current max, update the max
        if area > max_area:
            max_area = area
            max_cnt = cnt

    # Crop the bounding box with the maximum area from the input image and save it to a new file
    if max_cnt is not None:
        x, y, w, h = cv2.boundingRect(max_cnt)
        cropped = img[y:y+h, x:x+w]
        return cropped
    else:
        return img

In [45]:
def reshape_img(image, new_shape):
    resized = cv2.resize(image, new_shape)
    return resized

In [66]:
import os

benign_output_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape'
malignant_output_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/malignant_reshape'

benign_output_list = os.listdir(benign_output_path)
malignant_output_list = os.listdir(malignant_output_path)

image_benign_output = [f for f in benign_output_list if f.endswith('.jpg') or f.endswith('.png')]
image_malignant_output = [f for f in malignant_output_list if f.endswith('.jpg') or f.endswith('.png')]

benign_path = [benign_output_path + '/' + path for path in image_benign_output]
malignant_path = [malignant_output_path + '/' + path for path in image_malignant_output]

print(benign_path)

['/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1442.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/499.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1739.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1043.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1728.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1503.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/1604.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/610.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/383.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/881.jpg', '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/110.jpg', '/content/gdrive/MyDrive/

In [65]:
# reshape all output detection

# Set the input and output directory paths
input_dir = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign/'
output_dir = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/benign_reshape/'

# Set the new desired shape
new_shape = (128,128)

# Loop over all image files in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    image = cv2.imread(os.path.join(input_dir, filename))
    
    # Resize the image to the new shape
    resized = cv2.resize(image, new_shape)
    
    # Save the resized image to the output directory with the same filename
    cv2.imwrite(os.path.join(output_dir, filename), resized)

In [60]:
# reshape all output detection

# Set the input and output directory paths
input_dir = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/malignant/'
output_dir = '/content/gdrive/MyDrive/Skin_Cancer_Detection/OutputDetection/malignant_reshape/'

# Set the new desired shape
new_shape = (128,128)

# Loop over all image files in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    image = cv2.imread(os.path.join(input_dir, filename))
    
    # Resize the image to the new shape
    resized = cv2.resize(image, new_shape)
    
    # Save the resized image to the output directory with the same filename
    cv2.imwrite(os.path.join(output_dir, filename), resized)

In [67]:
img_benign = [cv2.imread(path) for path in benign_path]

X_benign = np.array(img_benign, dtype='uint8')

img_malignant = [cv2.imread(path) for path in malignant_path]

X_malignant = np.array(img_malignant, dtype='uint8')

In [68]:
print(X_benign.shape)
y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])

(1440, 128, 128, 3)


In [69]:
# Merge data 
X_train = np.concatenate((X_benign, X_malignant), axis = 0)
y_train = np.concatenate((y_benign, y_malignant), axis = 0)

In [70]:
# Shuffle data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

In [71]:
X_train = X_train/255.


In [72]:
# support vector machine classifier
from sklearn.svm import SVC

model = SVC()

model.fit(X_train.reshape(X_train.shape[0],-1), y_train)

SVC()

In [76]:
# predict

test_benign_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/Data_Skin_Cancer/data/test/benign'

benign_listdir = os.listdir(test_benign_path)

benign_path = [test_benign_path + '/' + path for path in benign_listdir]

y_pred = []
for path in benign_path:
    img = crop_object_detection(path)
    img_reshape = reshape_img(img, (128,128))

    X_test = np.array(img_reshape, dtype = 'uint8')
    X_test = X_test/255.
    # print(X_test.shape)

    pred = model.predict(X_test.reshape(1, 128*128*3))
    y_pred.append(pred)
    print(pred)

[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]


In [77]:
test_malignant_path = '/content/gdrive/MyDrive/Skin_Cancer_Detection/Data_Skin_Cancer/data/test/malignant'

malignant_listdir = os.listdir(test_malignant_path)

malignant_path = [test_malignant_path + '/' + path for path in malignant_listdir]

y_pred_mali = []
for path in malignant_path:
    img = crop_object_detection(path)
    img_reshape = reshape_img(img, (128,128))

    X_test = np.array(img_reshape, dtype = 'uint8')
    # print(X_test.shape)
    X_test = X_test/255.

    pred = model.predict(X_test.reshape(1, 128*128*3))
    y_pred_mali.append(pred)
    print(pred)

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]


In [78]:
y_pred = np.array(y_pred)
y_pred_mali = np.array(y_pred_mali)

In [80]:
print(((y_pred.shape[0]-np.sum(y_pred)) + np.sum(y_pred_mali))/(y_pred.shape[0]+ y_pred_mali.shape[0]) )

0.806060606060606
